#Load Data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision

from torchvision import *
from torch.utils.data import Dataset, DataLoader
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import time
import copy
import os

from models import *
from utils import progress_bar

batch_size = 64
learning_rate = 1e-3

transforms = transforms.Compose(
[
    transforms.ToTensor(),
    transforms.Resize([32, 32])
])

train_dataset = datasets.STL10(root='./data', split='train', download=True, transform=transforms)
test_dataset = datasets.STL10(root='./data', split='test', download=True, transform=transforms)


train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

images, labels = next(iter(train_dataloader)) 
print("images-size:", images.shape)

out = torchvision.utils.make_grid(images)
print("out-size:", labels.shape)

Files already downloaded and verified
Files already downloaded and verified
images-size: torch.Size([64, 3, 32, 32])
out-size: torch.Size([64])


In [2]:
class EarlyStopping():
    def __init__(self, tolerance=10, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if train_loss < self.min_delta:
            self.min_delta -= self.min_delta*0.5
            
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:  
                self.early_stop = True
        else:
            self.counter = 0

# Training

In [3]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(train_dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return train_loss


# Testing

In [4]:
def test(epoch,model_name):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(test_dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/'+model_name+'.pth')
        best_acc = acc
    return test_loss


In [5]:
classes = train_dataset.classes
lr = 0.01
n_epochs = 20

In [ ]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = DenseNet121()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=10, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'DenseNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1

In [ ]:
del net

In [6]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = PreActResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=10, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'PreActResNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1



Epoch: 0
 [===============================================================>.]  Step: 28ms | Tot: 3s652ms | Loss: 1.900 | Acc: 25.700% (1285/500 40/40 
 [===============================================================>.]  Step: 33ms | Tot: 3s803ms | Loss: 2.284 | Acc: 18.988% (1519/800 63/63 
Saving..

Epoch: 1
 [===============================================================>.]  Step: 19ms | Tot: 3s656ms | Loss: 1.606 | Acc: 36.780% (1839/500 40/40 
 [===============================================================>.]  Step: 33ms | Tot: 3s820ms | Loss: 2.007 | Acc: 29.663% (2373/800 63/63 
Saving..

Epoch: 2
 [===============================================================>.]  Step: 18ms | Tot: 3s651ms | Loss: 1.460 | Acc: 42.620% (2131/500 40/40  
 [===============================================================>.]  Step: 33ms | Tot: 3s866ms | Loss: 1.565 | Acc: 40.038% (3203/800 63/63 
Saving..

Epoch: 3
 [===============================================================>.]  Step: 16ms

In [ ]:
del net

In [7]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = DPN92()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=10, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'DPN') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 88ms | Tot: 22s393ms | Loss: 3.251 | Acc: 12.720% (636/500 79/79  
 [================================================================>]  Step: 82ms | Tot: 10s174ms | Loss: 3.968 | Acc: 17.350% (1388/800 125/125 
Saving..

Epoch: 1
 [================================================================>]  Step: 72ms | Tot: 22s402ms | Loss: 1.994 | Acc: 25.700% (1285/500 79/79  
 [================================================================>]  Step: 82ms | Tot: 10s177ms | Loss: 1.884 | Acc: 30.087% (2407/800 125/125 
Saving..

Epoch: 2
 [================================================================>]  Step: 78ms | Tot: 22s464ms | Loss: 1.750 | Acc: 30.760% (1538/500 79/79  
 [================================================================>]  Step: 82ms | Tot: 10s209ms | Loss: 1.604 | Acc: 37.013% (2961/800 125/125 
Saving..

Epoch: 3
 [================================================================>

In [ ]:
del net

In [10]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = EfficientNetB0()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=10, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'EffecientNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 49ms | Tot: 4s918ms | Loss: 1.948 | Acc: 26.080% (1304/500 79/79 
 [================================================================>]  Step: 32ms | Tot: 4s42ms | Loss: 2.416 | Acc: 34.987% (2799/800 125/125  
Saving..

Epoch: 1
 [================================================================>]  Step: 42ms | Tot: 5s120ms | Loss: 1.652 | Acc: 37.700% (1885/500 79/79 ===============================================================>.]  Step: 69ms | Tot: 5s78ms | Loss: 1.649 | Acc: 37.700% (1882/499 78/79 
 [================================================================>]  Step: 32ms | Tot: 4s45ms | Loss: 1.862 | Acc: 39.513% (3161/800 125/125  
Saving..

Epoch: 2
 [================================================================>]  Step: 41ms | Tot: 4s946ms | Loss: 1.437 | Acc: 46.480% (2324/500 79/79 
 [================================================================>]  Step: 31ms | Tot: 4s277ms | Lo

In [ ]:
del net

In [11]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = GoogLeNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=10, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'GoogleNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 67ms | Tot: 11s5ms | Loss: 1.779 | Acc: 30.620% (1531/500 79/79 79 
 [================================================================>]  Step: 68ms | Tot: 7s719ms | Loss: 1.658 | Acc: 37.138% (2971/800 125/125 
Saving..

Epoch: 1
 [================================================================>]  Step: 57ms | Tot: 11s196ms | Loss: 1.457 | Acc: 43.720% (2186/500 79/79  
 [================================================================>]  Step: 61ms | Tot: 7s714ms | Loss: 1.804 | Acc: 37.138% (2971/800 125/125 

Epoch: 2
 [================================================================>]  Step: 50ms | Tot: 11s22ms | Loss: 1.277 | Acc: 52.220% (2611/500 79/79 9 
 [================================================================>]  Step: 62ms | Tot: 7s656ms | Loss: 1.400 | Acc: 49.975% (3998/800 125/125 
Saving..

Epoch: 3
 [================================================================>]  Step: 46

In [ ]:
del net

In [9]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = ResNeXt29_2x64d()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=10, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'ResNext') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0


KeyboardInterrupt: 

In [ ]:
del net

In [12]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = DLA()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=10, min_delta=0.1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'DLA') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 37ms | Tot: 5s973ms | Loss: 1.872 | Acc: 26.100% (1305/500 79/79 
 [================================================================>]  Step: 39ms | Tot: 4s922ms | Loss: 1.972 | Acc: 27.188% (2175/800 125/125 
Saving..

Epoch: 1
 [================================================================>]  Step: 38ms | Tot: 5s989ms | Loss: 1.590 | Acc: 38.740% (1937/500 79/79 
 [================================================================>]  Step: 39ms | Tot: 4s958ms | Loss: 1.738 | Acc: 36.225% (2898/800 125/125 
Saving..

Epoch: 2
 [================================================================>]  Step: 35ms | Tot: 5s946ms | Loss: 1.444 | Acc: 44.200% (2210/500 79/79 
 [================================================================>]  Step: 40ms | Tot: 4s998ms | Loss: 1.695 | Acc: 39.438% (3155/800 125/125 
Saving..

Epoch: 3
 [================================================================>]  Step:

In [ ]:
del net